# 命名实体抽取
主要内容：
- 构建字典树，用于匹配命名实体
    - 用中/英文三元组的头实体构建前缀树（原先把 ILLs 也加入进来，但发现效果反而稍差）
    - 得到前缀树 `dict_en, dict_zh, dict`
    - 函数及初始化代码放在 `../src/prefixtree.jl` ，使用 `include` 导入
- 训练集识别结果，准确率 `99.4%`
    - 13992 个准确识别 | 变量名 `valids`
    - 65 个无法识别错误（以中文为主）| 变量名 `fails`
    - 20 个无法识别，有以下 7 种，除了前 2 个外，均为乱码问题 | 变量名 `nomatch`
    
     ```jl
     "who married the person of T-related eventhose Were the Days (song)?"
     "what is children of the person of T related eventhose Were the Days (song)?"
     "Savez-vous qu'avant produits liés à l'ENTITÉest?"
     "Savez-vous quels sont les pays participants"
     "Saviez-vous que la série du produits liés à l'ENTITÉest?"
     "Savez-vous ce qu’est la dynastie ?"
     "produits liés à l'ENTITÉbasé sur ce"
     ```
- 验证集识别情况：
    - 有 3 个无法识别，且均为问题错位导致，已在上一步单独处理
    - 提取 NER 后的数据放在 `ner_data/valid_ques_ner.txt`
    - 读取方式
    ```
    valid_ques_ner = Vector{String}(split(
        strip(read(open("ner_data/valid_data_ner.txt", "r"), String)), '\n'))
    ```

In [4]:
include("../src/extractdata.jl") # 导入 extract 中的数据
include("../src/prefixtree.jl") # 前缀树工具
mkpath("ner_data")

"ner_data"

In [5]:
# 检查准确性
valids, fails, nomatch = String[], String[], String[]
for (que, ner) in train_ques_ner
    sub = get_subject(que)
    isempty(sub) && (push!(nomatch, que); continue)
    sub == ner ? push!(valids, que) : push!(fails, que)
end
length.([valids, fails, nomatch]) # 13985, 72, 20

3-element Vector{Int64}:
 13992
    65
    20

In [14]:
# 准确率
13992 / length(train_ques_ner) * 100

99.39617816296085

In [13]:
# 验证集有效提取的数目
filter(i->isempty(get_subject(i)), valid_ques)

String[]

In [12]:
# 提取验证集
open("ner_data/valid_data_ner.txt", "w") do io
    for que in valid_ques
        println(io, que, '\t', get_subject(que))
    end
end